## 크롤링한 소요리문답자료를 형태소분석한다.

### 크롤링한 소요리문답 JSON파일을 읽어서 문장단위로 나누어 형태소 분석을 한다.
1. 소요리문답 JSON파일을 읽는다.
1. 질문과 답변을 문장단위로 나누어 형태소 분석을 한다.
1. 형태소 분석 결과를 후처리 한다.
1. 형태소 분석 결과를 파일로 저장한다.
    
### 형태소 분석 결과를 다시 JSON파일로 저장한다.

In [2]:
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
import itertools
import mmap
# from tqdm import tqdm_notebook
from tqdm import tqdm
import ujson
import re

def split_sentences(text):
    """주어진 텍스트를 문장 단위로 분절하여 돌려준다."""
    
    all_sentences = []
    lines = [line for line in text.strip().splitlines() if line.strip]
    
    for line in lines:
        sentences = re.split("(?<=[.?!]) ", line)
        all_sentences += sentences
    
    return all_sentences


def get_morph_anal(analyzer, text):
    """주어진 형태소 분석기 객체로 텍스트를 문장 단위로 형태소 분석하여 돌려준다."""
    
    sent_morph_anals = []
    sentences = split_sentences(text)
    
    for sentence in sentences:
        sent_morph_anal = analyzer.pos(sentence)
        sent_morph_anals.append(sent_morph_anal)
        
    return sent_morph_anals

def find_sublists(seq, sublist):
    length = len(sublist)
    for index, value in enumerate(seq):
        if value == sublist[0] and seq[index:index+length] == sublist:
            yield index, index+length
            

def replace_sublist(seq, target, replacement, maxreplace=None):
    sublists = find_sublists(seq, target)
    if maxreplace:
        sublists = itertools.islice(sublists, maxreplace)
    for start, end in sublists:
        seq[start:end] = replacement


def get_num_lines(file_path):
    """빠른 속도로 텍스트 파일의 줄 수를 세어 돌려준다.
    https://blog.nelsonliu.me/2016/07/29/progress-bars-for-python-file-reading-with-tqdm/
    """
    
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

def refine_answer(answer):
    """소요리문답의 성경구절이 있는 부분을 형태소분석을 위해 제거"""
    
    #print("소스:{}".format(answer))
    find_str = "(" 
    str_index = answer.find(find_str)
    if str_index > -1:
        answer_desc = answer[:str_index]
    else:
        answer_desc = answer
        
    refined_answer = answer_desc.translate({ ord('「'): '',ord('」'): ' ' })
        
    return refined_answer
    
def parse_row(row):
    """주어진 행을 열 단위로 분절하여 돌려준다."""

    qna_no = row["no"]
    question_source = row["question"]
    answer_source = row["answer"]
    
    return qna_no, question_source, answer_source


def compose_json_doc(qna_no, question_source, answer_source, refine_answer, question_ma, answer_ma):
    """주어진 문서 요소들로 JSON 문서를 생성하여 돌려준다."""
    
    json_doc = {
        "no": int(qna_no), 
        "question": question_source, 
        "answer": answer_source, 
        "refine_answer": refine_answer, 
        "question_ma": question_ma, 
        "answer_ma": answer_ma              
    }   
        
    return json_doc


def write_json_doc(output_file, json_doc):
    """주어진 JSON 문서를 출력 파일에 기록한다."""
        
    json_str = ujson.dumps(json_doc, ensure_ascii=False)
    print(json_str, file=output_file)

#=============================================================#
# 부분 리스트 치환에 의한 형태소 분석 결과 후처리
post_proc_pairs = [
    ([('신', 'XPN'), ('구약 성경', 'NNP')], [('신구약', 'NNG'), ('성경', 'NNG')]),
    ([('제', 'XPN'), ('일', 'NNB'), ('계명', 'NNG')], [('제 일 계명', 'NNG')]),
    ([('제', 'XPN'), ('이', 'MM'), ('계명', 'NNG')], [('제 이 계명', 'NNG')]),
    ([('제', 'XPN'), ('삼', 'NR'), ('계명', 'NNG')], [('제 삼 계명', 'NNG')]),
    ([('제', 'XPN'), ('사', 'NNG'), ('계명', 'NNG')], [('제 사 계명', 'NNG')]),
    ([('제', 'XPN'), ('오', 'NR'), ('계명', 'NNG')], [('제 오 계명', 'NNG')]),
    ([('제', 'XPN'), ('육', 'NR'), ('계명', 'NNG')], [('제 육 계명', 'NNG')]),
    ([('제', 'XPN'), ('치', 'VV'), ('ㄹ', 'ETM'), ('계명', 'NNG')], [('제 칠 계명', 'NNG')]),
    ([('제', 'XPN'), ('팔', 'VV'), ('ㄹ', 'ETM'), ('계명', 'NNG')], [('제 팔 계명', 'NNG')]),
    ([('제', 'XPN'), ('구', 'NNG'), ('계명', 'NNG')], [('제 구 계명', 'NNG')]),
    ([('제', 'XPN'), ('십', 'NR'), ('계명', 'NNG')], [('제 십 계명', 'NNG')]),
    ([('첫','MM'), ('말씀','NNG')], [('첫','NR'),('말씀','NNG')]),
    ([('첫','MM'), ('기도','NNG')], [('첫','NR'),('기도','NNG')])
]


input_file_name = "data/crawling/catechism.txt"
output_file_name = "data/catechism/catechism.ma.txt"
komoran = Komoran()
hannanum = Hannanum()


with open(input_file_name, "r", encoding="utf-8") as input_file, \
        open(output_file_name, "w", encoding="utf-8") as output_file:
        
    for line in tqdm(input_file, desc="Reading documents", 
                                  total=get_num_lines(input_file_name)):
        
        doc_row = ujson.loads(line)
        qna_no, question_source, answer_source = parse_row(doc_row)
        refined_answer = refine_answer(answer_source)        

        #print("질문[{}]:{}".format(qna_no, question_source.translate({ ord('「'): '',ord('」'): ' ' })))
        #print("답변:{}".format(refined_answer))
        
        question_ma = get_morph_anal(komoran, question_source.translate({ ord('「'): '',ord('」'): ' ' }) )
        # 소요리 문답 질문에 대한 부분 리스트 치환에 의한 형태소 분석 결과 후처리
        for src, dst in post_proc_pairs:
            replace_sublist(question_ma[0], src, dst)

        answer_ma = get_morph_anal(komoran, refined_answer)
        # 소요리 문답 답변에 대한 부분 리스트 치환에 의한 형태소 분석 결과 후처리
        for src, dst in post_proc_pairs:
            replace_sublist(answer_ma[0], src, dst)

        print("질문[{}]:{}".format(qna_no, question_ma[0]))
        print("답변:{}".format(answer_ma[0]))
        print("{}".format("="*80))
        json_doc = compose_json_doc(qna_no, question_source, answer_source, refined_answer, question_ma[0], answer_ma[0])
        write_json_doc(output_file, json_doc)



Reading documents:  21%|████████████▌                                                | 22/107 [00:00<00:00, 206.16it/s]

질문[1]:[('사람', 'NNG'), ('의', 'JKG'), ('제일', 'NNG'), ('되', 'VV'), ('는', 'ETM'), ('목적', 'NNG'), ('이', 'JKS'), ('무엇', 'NP'), ('이', 'VCP'), ('ㄴ가', 'EF'), ('?', 'SF')]
답변:[('사람', 'NNG'), ('의', 'JKG'), ('제일', 'NNG'), ('되', 'VV'), ('는', 'ETM'), ('목적', 'NNG'), ('은', 'JX'), ('하나님', 'NNG'), ('을', 'JKO'), ('영화', 'NNG'), ('롭', 'XSA'), ('게', 'EC'), ('하', 'VV'), ('는', 'ETM'), ('것', 'NNB'), ('과', 'JKB'), ('영원', 'NNG'), ('하', 'XSV'), ('도록', 'EC'), ('그', 'NP'), ('를', 'JKO'), ('즐거워하', 'VV'), ('는', 'ETM'), ('것', 'NNB'), ('이', 'VCP'), ('다', 'EF'), ('.', 'SF')]
질문[2]:[('하나님', 'NNP'), ('께서', 'JKS'), ('무슨', 'MM'), ('규칙', 'NNG'), ('을', 'JKO'), ('우리', 'NP'), ('에게', 'JKB'), ('주', 'VX'), ('시', 'EP'), ('어', 'EC'), ('어떻', 'VA'), ('게', 'EC'), ('자기', 'NP'), ('를', 'JKO'), ('영화', 'NNG'), ('롭', 'XSA'), ('게', 'EC'), ('하', 'VV'), ('고', 'EC'), ('즐거워하', 'VV'), ('ㄹ', 'ETM'), ('것', 'NNB'), ('을', 'JKO'), ('지시', 'NNG'), ('하', 'XSV'), ('시', 'EP'), ('었', 'EP'), ('는가', 'EF'), ('?', 'SF')]
답변:[('신구약', 'NNG'), ('성경', 'NNP'), ('에', '

Reading documents:  60%|████████████████████████████████████▍                        | 64/107 [00:00<00:00, 204.19it/s]

질문[39]:[('하나님', 'NNP'), ('께서', 'JKS'), ('사람', 'NNG'), ('에게', 'JKB'), ('요구', 'NNG'), ('하', 'XSV'), ('시', 'EP'), ('는', 'ETM'), ('본분', 'NNG'), ('이', 'JKS'), ('무엇', 'NP'), ('이', 'VCP'), ('ㄴ가', 'EF'), ('?', 'SF')]
답변:[('하나님', 'NNP'), ('께서', 'JKS'), ('사람', 'NNG'), ('에게', 'JKB'), ('요구', 'NNG'), ('하', 'XSV'), ('시', 'EP'), ('는', 'ETM'), ('본분', 'NNG'), ('은', 'JX'), ('그', 'MM'), ('나타내', 'VV'), ('어', 'EC'), ('보이', 'VV'), ('시', 'EP'), ('ㄴ', 'ETM'), ('뜻', 'NNG'), ('을', 'JKO'), ('복종', 'NNG'), ('하', 'XSV'), ('는', 'ETM'), ('것', 'NNB'), ('이', 'VCP'), ('다', 'EF'), ('.', 'SF')]
질문[40]:[('하나님', 'NNP'), ('께서', 'JKS'), ('자기', 'NP'), ('에게', 'JKB'), ('복종', 'NNG'), ('하', 'XSV'), ('ㄹ', 'ETM'), ('규칙', 'NNG'), ('으로', 'JKB'), ('사람', 'NNG'), ('에게', 'JKB'), ('처음', 'NNG'), ('나타내', 'VV'), ('어', 'EC'), ('보이', 'VV'), ('시', 'EP'), ('ㄴ', 'ETM'), ('것', 'NNB'), ('이', 'JKS'), ('무엇', 'NP'), ('이', 'VCP'), ('ㄴ가', 'EF'), ('?', 'SF')]
답변:[('하나님', 'NNP'), ('께서', 'JKS'), ('자기', 'NP'), ('에게', 'JKB'), ('복종', 'NNG'), ('하', 'XSV'), ('ㄹ'

Reading documents:  81%|█████████████████████████████████████████████████▌           | 87/107 [00:00<00:00, 209.73it/s]


답변:[('타락', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETM'), ('후', 'NNG'), ('로', 'JKB'), ('사람', 'NNG'), ('만', 'JX'), ('으로', 'JKB'), ('는', 'JX'), ('금', 'NNG'), ('생', 'XSN'), ('에서', 'JKB'), ('하나님', 'NNP'), ('의', 'JKG'), ('계명', 'NNP'), ('을', 'JKO'), ('완전히', 'MAG'), ('지키', 'VV'), ('ㄹ', 'ETM'), ('수', 'NNB'), ('없', 'VA'), ('고', 'EC'), ('날', 'NNG'), ('마다', 'JX'), ('생각', 'NNG'), ('과', 'JC'), ('말', 'NNG'), ('과', 'JC'), ('행동', 'NNG'), ('으로써', 'JKB'), ('범하', 'VV'), ('ㄴ다', 'EF'), ('.', 'SF')]
질문[83]:[('법', 'NNG'), ('을', 'JKO'), ('범하', 'VV'), ('ㄴ', 'ETM'), ('모든', 'MM'), ('죄', 'NNG'), ('가', 'JKS'), ('다', 'MAG'), ('같이', 'MAG'), ('악하', 'VA'), ('ㄴ가', 'EF'), ('?', 'SF')]
답변:[('어떠', 'XR'), ('하', 'XSA'), ('ㄴ', 'ETM'), ('죄', 'NNG'), ('는', 'JX'), ('그', 'MM'), ('본질', 'NNG'), ('과', 'JC'), ('여러', 'MM'), ('가지', 'NNB'), ('얽히', 'VV'), ('ㄴ', 'ETM'), ('끝', 'NNG'), ('이', 'JKS'), ('있', 'VV'), ('으므로', 'EC'), ('하나님', 'NNP'), ('앞', 'NNG'), ('에서', 'JKB'), ('다르', 'VA'), ('ㄴ', 'ETM'), ('죄', 'NNG'), ('보다', 'JKB'), ('더', 'MAG'), ('악하', '

Reading documents: 100%|█████████████████████████████████████████████████████████████| 107/107 [00:01<00:00, 60.65it/s]

질문[102]:[('둘째', 'NR'), ('기도', 'NNG'), ('에', 'JKB'), ('우리', 'NP'), ('가', 'JKS'), ('무엇', 'NP'), ('을', 'JKO'), ('구하', 'VV'), ('는가', 'EF'), ('?', 'SF')]
답변:[('주기', 'NNP'), ('도문', 'NNP'), ('의', 'JKG'), ('둘째', 'NR'), ('기도', 'NNG'), ('에', 'JKB'), ('나라', 'NNG'), ('이', 'NNP'), ('임하', 'VV'), ('옵', 'EP'), ('소서', 'EC'), ('하', 'VV'), ('ㅁ', 'ETN'), ('은', 'JX'), ('사단', 'NNP'), ('의', 'JKG'), ('나라', 'NNG'), ('가', 'JKS'), ('멸망', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('은혜', 'NNP'), ('의', 'JKG'), ('나라', 'NNG'), ('가', 'JKS'), ('흥', 'NNG'), ('왕', 'NNG'), ('하', 'XSV'), ('아', 'EC'), ('우리', 'NP'), ('와', 'JKB'), ('다른', 'MM'), ('사람', 'NNG'), ('으로', 'JKB'), ('하여금', 'MAG'), ('그리로', 'MAG'), ('들어가', 'VV'), ('아', 'EC'), ('항상', 'MAG'), ('있', 'VV'), ('게', 'EC'), ('하', 'VV'), ('시', 'EP'), ('고', 'EC'), ('또', 'MAJ'), ('영광', 'NNP'), ('의', 'JKG'), ('나라', 'NNG'), ('가', 'JKS'), ('속히', 'MAG'), ('임하', 'VV'), ('게', 'EC'), ('하', 'VV'), ('옵', 'EP'), ('시', 'EP'), ('ㅁ', 'ETN'), ('을', 'JKO'), ('구하', 'VV'), ('는', 'ETM'), ('것', 'NNB'), ('